# New featurizer

This code implements the new featurizer of code and demonstrates its usage. It was used originally to featurize lines from Gerrit's comments.

This version contains extra comments useful for understanding how this works. 

(c) Miroslaw Staron

## Class declaration

The class FeatureMaker is a code that takes a given set of features and a line as input. As output it produces the feature vector. 

It's supposed to be used as a utility and does not have much code in it. The main function is featurize. It has three variants, which have different performance, but the one that is just called featurize is best for most cases. 

The other class - DataSet - is a class which contains the feature vectors for all lines in a specific piece of code. Kind of like a DataFrame, but specific for this purpose.

In [7]:
# This is a python program to create feature vectors from the programming language code
# in such a way that every distinct line is indeed distinct

# author: Miroslaw Staron

import re
import pandas as pd
import time
from collections import Counter
import re

class FeatureMaker:
    """Class makes a feature vector base on the set of features defined in the class parameters"""

    def __init__(self):
        """The initial state of the feature vector is actually an empty vector"""
        self.featureVector = []

    def addNewFeature(self, newFeature):
        """Adds just one feature to the list of features"""
        self.featureVector.append(newFeature)

    def addNewFeatures(self, newFeatures):
        """Adds a list of features"""
        self.featureVector = self.featureVector + newFeatures
    
    def getFeatureVector(self):
        return self.featureVector

    def featurize(self, line):
        """Counts the frequency of each feature in a given line"""
        self.features = []
        for feature in self.featureVector:
            self.features.append(str(line.count(feature)))
        return self.features
    
    def featurize2(self, line):
        """Counts the frequency of each feature in a given line"""
        self.features = []
        counter = Counter(line)
        for feature in self.featureVector:
            self.features.append(str(counter[feature]))
        return self.features
    
    def featurize3(self, line):
        """Counts the frequency of each feature in a given line"""
        self.features = []
        for feature in self.featureVector:
            self.features.append(str(len(re.findall(str(feature), line))))
        return self.features
    
    def featuresToString(self):
        strFeatures = '$'.join(str(e) for e in self.features)
        return strFeatures
    
    def findNewFeatures(self, lstTokens):
        newElements = list(set(lstTokens) - set(self.featureVector))
        return newElements

def tokenizeString(myString):
    """This function takes a line and returns a set of strings; empty strings are removed"""
    tokenList = '[\(|"|,|.|;|\)|\[|\]|{|}| ,|\n|\t:]'
    tokens = re.split(tokenList, myString)
    tokens = list(filter(None, tokens))
    return tokens

class DataSet:
    """The class makes the connection between the code and its features"""

    def __init__(self):
        self.dictRows = {}
        self.featureVector = []
    
    def addFeatureVector(self, lstFeatureVector):
        self.featureVector = lstFeatureVector

    def addNewLine(self, strLine, lstFeatures):
        self.dictRows[lstFeatures] = strLine 

    def hasLine(self, lstFeatures):
        return (lstFeatures in self.dictRows.keys())
            
    def getLine(self, lstFeatures):
        return self.dictRows[lstFeatures]

    def toCSV(self, strFilename):
        fFile = open(strFilename, 'w', encoding='utf8')
        strFirstLine = 'line$'
        strFirstLine += '$'.join(self.featureVector) + '\n'
        fFile.write(strFirstLine)
        for key, value in self.dictRows.items():
            value = value.replace("\n", "").replace("$","").replace("\r","").replace("\t","")
            strToFile = f'{value}${key}\n'
            fFile.write(strToFile)
        fFile.close()
    
    def flush(self):
        self.dictRows = {}
        self.featureVector = []

## The feature finder function

This function is the main algorithm. It takes a list of lines to featurize and a name of the file where the list of featurers is to be stored. The file is needed as the list of features is saved periodically. 

The periodical saving is needed as the algorithm can take weeks to find the optimal feature set (for code bases that have over 100,000 loc) and sometimes the computer may restart during that time. To prevent loosing the work, the feature set is saved periodically. 

In [9]:

# this is an recursive function to find a set of features for a set of lines
# the function goes through all lines and then checks which lines are identical given the feature list
# then it takes one of the tokens from the identical lines and makes a recursion
def findFeatureListIterative(lstLines, strOutputFeatureFile):
    #print(f'Lines: {len(lstLines)}, features: {len(lstFeatures)}')
    start_time = time.time()
    
    featurizer = FeatureMaker()
    featureList = []

    if len(lstLines) > 0:
        initialFeatures = tokenizeString(' '.join(lstLines))
        featurizer.addNewFeature(initialFeatures[0])
        featureList.append(initialFeatures[0])
    else:
        return featureList

    featureAdded = True

    while featureAdded:
        dictLinesUnique = {}
        lstNotUnique = []
        featureAdded = False

        # featurizing all lines in this iteration
        for line in lstLines:
            mFeatures = featurizer.featurize(line)
            strFeatures = '$'.join(mFeatures)
            if not (strFeatures in dictLinesUnique.keys()):
                dictLinesUnique[strFeatures] = line
            else:
                lstNotUnique.append(line)
                lstNotUnique.append(dictLinesUnique[strFeatures])
        
        lstNotUnique = list(set(lstNotUnique))
        strTime = f'{(time.time() - start_time):.2f} sec.'
        start_time = time.time()
        print(f'Non-unique lines remaining: {len(lstNotUnique)}, features found: {len(featureList)} in {strTime}')
        
        # this if statement is used only to save the features
        if len(featureList) % 10 == 0:
            print('Saving feature list...')
            fFile = open(strOutputFeatureFile, 'w', encoding='utf8')
            strFirstLine = str(len(lstNotUnique))
            strFirstLine = '$'.join(featureList) + '\n'
            fFile.write(strFirstLine)
            fFile.close()
            print('Done...')

        # and kicking-off the next iteration if necessary
        # by necessary I mean that there are lines that are not different
        if len(lstNotUnique) > 0:
            allLines = lstNotUnique
            getTokens = tokenizeString(' '.join(allLines))
            #getTokens.sort(reverse=True)
            for oneToken in getTokens:
                if not oneToken in featureList:                    
                    featureList.append(oneToken)
                    featurizer.addNewFeature(oneToken)
                    featureAdded = True
                    break
            if featureAdded:
                #featureList = findFeatureList(allLines, featureList)
                lstLines = lstNotUnique
        
    # returning the feature list
    return featureList

## Demonstration of how to use it

This code demonstrates how to use the featurizer. 

It first reads the code (in this case as a Pandas data frame), then it processes it so that we have a list of lines of code only. Then it removes the duplicated lines (to make the algorithm work faster). 

In [3]:
##
## This is a block where we test the featurizer class and the dataset class
##

strOutputFile = './featurized_lines.csv'

dfCode = pd.read_csv('./55train-lines.csv', 
                    sep='$', 
                    error_bad_lines=False, 
                    warn_bad_lines=True, 
                    header=0, 
                    index_col=False)

mLines = [line for line in dfCode['contents'] if str(line) != 'nan' ]

print(f'All Lines: {len(mLines)}')

mLines = list(set(mLines))

print(f'Unique Lines: {len(mLines)}')
#print(mLines)



C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3018: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


All Lines: 1394593
Unique Lines: 140286


This line executes the feature finder and gets the set of features as a result. 

The feature finder function prints out a bit of information about how much work it has left and how much time it takes per line. The time grows as the number of features grows with each iteration. Since the FeatureMaker.featurize function is a loop, it takes more time for each iteration. 

In [10]:
features = findFeatureListIterative(mLines, './feature_list.csv')

Non-unique lines remaining: 140285, features found: 1 in 0.66 sec.
Non-unique lines remaining: 140285, features found: 2 in 0.74 sec.
Non-unique lines remaining: 140279, features found: 3 in 0.81 sec.
Non-unique lines remaining: 140268, features found: 4 in 0.85 sec.
Non-unique lines remaining: 140267, features found: 5 in 0.96 sec.
Non-unique lines remaining: 140261, features found: 6 in 0.99 sec.
Non-unique lines remaining: 140256, features found: 7 in 1.05 sec.
Non-unique lines remaining: 140248, features found: 8 in 1.09 sec.
Non-unique lines remaining: 140246, features found: 9 in 1.13 sec.
Non-unique lines remaining: 140238, features found: 10 in 1.12 sec.
Saving feature list...
Done...
Non-unique lines remaining: 140233, features found: 11 in 1.13 sec.
Non-unique lines remaining: 140163, features found: 12 in 1.18 sec.
Non-unique lines remaining: 140162, features found: 13 in 1.22 sec.
Non-unique lines remaining: 140140, features found: 14 in 1.27 sec.
Non-unique lines remaining

Non-unique lines remaining: 116258, features found: 117 in 5.94 sec.
Non-unique lines remaining: 116258, features found: 118 in 5.88 sec.
Non-unique lines remaining: 116256, features found: 119 in 6.01 sec.
Non-unique lines remaining: 115383, features found: 120 in 6.00 sec.
Saving feature list...
Done...
Non-unique lines remaining: 115343, features found: 121 in 6.02 sec.
Non-unique lines remaining: 115343, features found: 122 in 6.00 sec.
Non-unique lines remaining: 115330, features found: 123 in 6.09 sec.
Non-unique lines remaining: 114962, features found: 124 in 6.15 sec.
Non-unique lines remaining: 113542, features found: 125 in 6.11 sec.
Non-unique lines remaining: 113541, features found: 126 in 6.11 sec.
Non-unique lines remaining: 113540, features found: 127 in 6.23 sec.
Non-unique lines remaining: 113535, features found: 128 in 6.19 sec.
Non-unique lines remaining: 113534, features found: 129 in 6.35 sec.
Non-unique lines remaining: 112404, features found: 130 in 6.33 sec.
Sav

Non-unique lines remaining: 94083, features found: 231 in 8.95 sec.
Non-unique lines remaining: 93570, features found: 232 in 8.96 sec.
Non-unique lines remaining: 93557, features found: 233 in 8.98 sec.
Non-unique lines remaining: 93538, features found: 234 in 8.83 sec.
Non-unique lines remaining: 93483, features found: 235 in 9.02 sec.
Non-unique lines remaining: 93476, features found: 236 in 9.04 sec.
Non-unique lines remaining: 93448, features found: 237 in 9.09 sec.
Non-unique lines remaining: 93448, features found: 238 in 9.24 sec.
Non-unique lines remaining: 93157, features found: 239 in 9.12 sec.
Non-unique lines remaining: 93144, features found: 240 in 9.20 sec.
Saving feature list...
Done...
Non-unique lines remaining: 92910, features found: 241 in 9.20 sec.
Non-unique lines remaining: 92910, features found: 242 in 9.19 sec.
Non-unique lines remaining: 92670, features found: 243 in 9.26 sec.
Non-unique lines remaining: 90644, features found: 244 in 9.23 sec.
Non-unique lines 

Non-unique lines remaining: 85036, features found: 346 in 12.14 sec.
Non-unique lines remaining: 85036, features found: 347 in 12.26 sec.
Non-unique lines remaining: 85032, features found: 348 in 12.37 sec.
Non-unique lines remaining: 84869, features found: 349 in 12.26 sec.
Non-unique lines remaining: 84797, features found: 350 in 12.30 sec.
Saving feature list...
Done...
Non-unique lines remaining: 84790, features found: 351 in 12.57 sec.
Non-unique lines remaining: 84787, features found: 352 in 12.35 sec.
Non-unique lines remaining: 84784, features found: 353 in 12.34 sec.
Non-unique lines remaining: 84550, features found: 354 in 12.50 sec.
Non-unique lines remaining: 84548, features found: 355 in 12.65 sec.
Non-unique lines remaining: 84539, features found: 356 in 12.45 sec.
Non-unique lines remaining: 84456, features found: 357 in 12.42 sec.
Non-unique lines remaining: 84456, features found: 358 in 12.51 sec.
Non-unique lines remaining: 84456, features found: 359 in 12.41 sec.
Non

Non-unique lines remaining: 74158, features found: 460 in 13.65 sec.
Saving feature list...
Done...
Non-unique lines remaining: 74157, features found: 461 in 13.67 sec.
Non-unique lines remaining: 74149, features found: 462 in 13.68 sec.
Non-unique lines remaining: 74148, features found: 463 in 13.70 sec.
Non-unique lines remaining: 74113, features found: 464 in 13.64 sec.
Non-unique lines remaining: 74111, features found: 465 in 13.70 sec.
Non-unique lines remaining: 74109, features found: 466 in 13.84 sec.
Non-unique lines remaining: 74084, features found: 467 in 14.02 sec.
Non-unique lines remaining: 73106, features found: 468 in 14.02 sec.
Non-unique lines remaining: 73103, features found: 469 in 13.66 sec.
Non-unique lines remaining: 72934, features found: 470 in 13.65 sec.
Saving feature list...
Done...
Non-unique lines remaining: 72872, features found: 471 in 13.60 sec.
Non-unique lines remaining: 72870, features found: 472 in 13.86 sec.
Non-unique lines remaining: 72859, featur

...

Non-unique lines remaining: 23018, features found: 13254 in 105.36 sec.
Non-unique lines remaining: 23018, features found: 13255 in 105.48 sec.
Non-unique lines remaining: 23018, features found: 13256 in 105.50 sec.
Non-unique lines remaining: 23018, features found: 13257 in 105.67 sec.
Non-unique lines remaining: 23018, features found: 13258 in 105.39 sec.
Non-unique lines remaining: 23018, features found: 13259 in 106.33 sec.
Non-unique lines remaining: 23018, features found: 13260 in 105.21 sec.
Saving feature list...
Done...
Non-unique lines remaining: 23018, features found: 13261 in 106.49 sec.
Non-unique lines remaining: 23018, features found: 13262 in 105.66 sec.
Non-unique lines remaining: 23018, features found: 13263 in 105.73 sec.
Non-unique lines remaining: 23018, features found: 13264 in 105.60 sec.
Non-unique lines remaining: 23018, features found: 13265 in 105.65 sec.
Non-unique lines remaining: 23018, features found: 13266 in 105.62 sec.
Non-unique lines remaining: 23018

Non-unique lines remaining: 22992, features found: 13364 in 106.49 sec.
Non-unique lines remaining: 22992, features found: 13365 in 106.40 sec.
Non-unique lines remaining: 22992, features found: 13366 in 106.75 sec.
Non-unique lines remaining: 22992, features found: 13367 in 106.56 sec.
Non-unique lines remaining: 22992, features found: 13368 in 106.48 sec.
Non-unique lines remaining: 22992, features found: 13369 in 106.62 sec.
Non-unique lines remaining: 22992, features found: 13370 in 106.45 sec.
Saving feature list...
Done...
Non-unique lines remaining: 22990, features found: 13371 in 106.54 sec.
Non-unique lines remaining: 22990, features found: 13372 in 106.69 sec.
Non-unique lines remaining: 22990, features found: 13373 in 106.62 sec.
Non-unique lines remaining: 22990, features found: 13374 in 107.36 sec.
Non-unique lines remaining: 22990, features found: 13375 in 106.79 sec.
Non-unique lines remaining: 22990, features found: 13376 in 107.18 sec.
Non-unique lines remaining: 22990

Non-unique lines remaining: 22975, features found: 13474 in 111.05 sec.
Non-unique lines remaining: 22975, features found: 13475 in 110.86 sec.
Non-unique lines remaining: 22975, features found: 13476 in 111.03 sec.
Non-unique lines remaining: 22975, features found: 13477 in 110.52 sec.
Non-unique lines remaining: 22975, features found: 13478 in 112.62 sec.
Non-unique lines remaining: 22975, features found: 13479 in 118.26 sec.
Non-unique lines remaining: 22975, features found: 13480 in 116.65 sec.
Saving feature list...
Done...
Non-unique lines remaining: 22975, features found: 13481 in 115.35 sec.
Non-unique lines remaining: 22975, features found: 13482 in 114.81 sec.
Non-unique lines remaining: 22975, features found: 13483 in 115.57 sec.
Non-unique lines remaining: 22975, features found: 13484 in 115.89 sec.
Non-unique lines remaining: 22975, features found: 13485 in 117.11 sec.
Non-unique lines remaining: 22975, features found: 13486 in 116.34 sec.
Non-unique lines remaining: 22975

Non-unique lines remaining: 22957, features found: 13584 in 110.17 sec.
Non-unique lines remaining: 22957, features found: 13585 in 109.85 sec.
Non-unique lines remaining: 22957, features found: 13586 in 109.90 sec.
Non-unique lines remaining: 22957, features found: 13587 in 109.94 sec.
Non-unique lines remaining: 22957, features found: 13588 in 109.95 sec.
Non-unique lines remaining: 22957, features found: 13589 in 109.80 sec.
Non-unique lines remaining: 22957, features found: 13590 in 110.21 sec.
Saving feature list...
Done...
Non-unique lines remaining: 22957, features found: 13591 in 110.45 sec.
Non-unique lines remaining: 22957, features found: 13592 in 109.88 sec.
Non-unique lines remaining: 22957, features found: 13593 in 110.16 sec.
Non-unique lines remaining: 22957, features found: 13594 in 110.10 sec.
Non-unique lines remaining: 22957, features found: 13595 in 110.29 sec.
Non-unique lines remaining: 22957, features found: 13596 in 110.05 sec.
Non-unique lines remaining: 22957

Non-unique lines remaining: 22926, features found: 13694 in 133.32 sec.
Non-unique lines remaining: 22926, features found: 13695 in 132.23 sec.
Non-unique lines remaining: 22926, features found: 13696 in 132.87 sec.
Non-unique lines remaining: 22926, features found: 13697 in 132.49 sec.
Non-unique lines remaining: 22926, features found: 13698 in 132.75 sec.
Non-unique lines remaining: 22926, features found: 13699 in 132.08 sec.
Non-unique lines remaining: 22926, features found: 13700 in 133.88 sec.
Saving feature list...
Done...
Non-unique lines remaining: 22925, features found: 13701 in 133.01 sec.
Non-unique lines remaining: 22925, features found: 13702 in 133.61 sec.
Non-unique lines remaining: 22923, features found: 13703 in 132.68 sec.
Non-unique lines remaining: 22922, features found: 13704 in 132.71 sec.


# Function to featurize the code based on a predefined list of features

This function goes through the code once again and creates the feature vector for each line. It is separate from the previous one as we can use any feature list we want, not only the one from this new featurizer. 

In [11]:

def featurizeListPredefined(lstLines, lstFeatures):
    dtLines = DataSet()
    
    featurizer = FeatureMaker()

    featurizer.addNewFeatures(lstFeatures)

    foundNewFeature = True
    i = 1
    while foundNewFeature:
        foundNewFeature = False
        print(f'Pass number: {i}')
        i += 1
        iLine = 0
        total = len(lstLines)
        for line in lstLines:
            iLine += 1
            if not foundNewFeature: 
                mFeatures = featurizer.featurize(line)  
                if not all(v == 0 for v in mFeatures):    
                    strFeatures = featurizer.featuresToString()
                    if not dtLines.hasLine(strFeatures):
                        dtLines.addNewLine(line, strFeatures)
                    else:
                        strLine = dtLines.getLine(strFeatures)
                        if strLine != line:
                            lineTokens = tokenizeString(line)
                            oldLineTokens = tokenizeString(strLine)
                            newFeatures = featurizer.findNewFeatures(lineTokens+oldLineTokens)
                            if len(newFeatures) > 0:
                                featurizer.addNewFeature(newFeatures[0])
                                foundNewFeature = True
                                dtLines.flush()
                                dtLines.addFeatureVector(featurizer.featureVector)
                                print(f'Found new feature at line {iLine} of {total}')                           

    return dtLines

## Demonstration of featurizing the code

In this piece of code, we just use the featurizer. We already have a list of features to use and now we need to find the feature vector for each line. 

This function takes a while, mostly saving the feature table to CSV, which tends to be very big. If you want to check the progress, you can take a look at the memory consumption here or the size of the output file on the disk. 

In [14]:
strOutputFile = './featurized_lines.csv'
dtLines = featurizeListPredefined(mLines, features)
dtLines.addFeatureVector(features)
dtLines.toCSV(strOutputFile)

Pass number: 1
